In [ ]:
# We're going to install this module to help us parse datetimes from the raw dataset
!pip install dateparser

In [24]:
from pymongo import MongoClient, InsertOne, UpdateOne
import pprint
import dateparser
from bson.json_util import loads
import time


In [25]:
# Replace XXXX with your connection URI from the Atlas UI
client = MongoClient("localhost",27017)
people_raw = client.cleansing['people-raw']

In [26]:
batch_size = 1000
inserts = []
count = 0

In [16]:
# There are over 50,000 lines, so this might take a while...
# Make sure to wait until the cell finishes executing before moving on (the * will turn into a number)
with open("./people-raw.json") as dataset:
    for line in dataset:
        doc = loads(line)
        update = UpdateOne({'_id': doc['_id']}, {'$set': doc}, upsert=True)
        inserts.append(update)

        count += 1

        if count == batch_size:
            people_raw.bulk_write(inserts)
            inserts = []
            count = 0

if inserts:
    people_raw.bulk_write(inserts)
    count = 0


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: cleansing.people-raw index: _id_ dup key: { _id: ObjectId('57d7a121fa937f710a7d486e') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('57d7a121fa937f710a7d486e')}, 'op': {'_id': ObjectId('57d7a121fa937f710a7d486e'), 'last_name': 'Pham', 'quote': 'Aliquam est reiciendis alias neque ad.', 'job': 'Counselling psychologist', 'ssn': '401-31-6615', 'address': {'city': 'Burgessborough', 'street': '83248 Woods Extension', 'zip': '47201'}, 'first_name': 'Yvonne', 'company_id': ObjectId('57d7a121fa937f710a7d486d'), 'employer': 'Terry and Sons', 'birthday': datetime.datetime(2011, 3, 17, 11, 21, 36), 'email': 'murillobrian@cox.net'}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

In [33]:
# people_raw = '''{"_id": {"$oid": "57d7a122fa937f710a7d4a4a"}, "last_name": "Hayes", "quote": "Voluptates perferendis repellendus dolorem.", "job": "Photographer", "ssn": "107-02-2505", "address": {"city": "Lake Kimberly", "state": "Florida", "street": "3412 John Light", "zip": "15036-8821"}, "first_name": "Beth", "company_id": {"$oid": "57d7a122fa937f710a7d49f4"}, "employer": "Martin-Johnson", "birthday": {"$date": "2013-02-10T12:14:06.000Z"}, "email": "ajones@hansen.com"}
# {"_id": {"$oid": "57d7a122fa937f710a7d4a4b"}, "last_name": "Ross", "quote": "Reiciendis omnis porro qui ea maiores assumenda doloremque.", "job": "Occupational hygienist", "ssn": "201-55-2105", "address": {"city": "Lake Kimberly", "state": "Florida", "street": "522 Joseph Inlet Apt. 618", "zip": "15036-8821"}, "first_name": "Justin", "company_id": {"$oid": "57d7a122fa937f710a7d49f4"}, "employer": "Martin-Johnson", "birthday": "Sunday, December 8, 2013 at 6:18:02 AM", "email": "mia54@gmail.com"}'''
count = 0
string_count = 0
for doc in people_raw.find():
    birthday = doc["birthday"]
    if isinstance(birthday, str):
        # Перетворення рядкового значення у дату
        birthday = time.strptime(birthday, '%A, %B %d, %Y at %I:%M:%S %p')
        string_count += 1
    # print(birthday)
    count += 1
print(count)
print(string_count)

50474
10382


In [29]:
# Confirm that 50,474 documents are in your collection before moving on
people_raw.count()

TypeError: 'Collection' object is not callable. If you meant to call the 'count' method on a 'Collection' object it is failing because no such method exists.

In [ ]:
# Replace YYYY with a query on the people-raw collection that will return a cursor with only
# documents where the birthday field is a string
people_with_string_birthdays = people_raw.find({'birthday':{'$type':'string'}})

In [ ]:
# This is the answer to verify you completed the lab
people_with_string_birthdays.count()

In [ ]:
updates = []
# Again, we're updating several thousand documents, so this will take a little while
for person in people_with_string_birthdays:
    # Pymongo converts datetime objects into BSON Dates. The dateparser.parse function returns a
    # datetime object, so we can simply do the following to update the field properly.
    # Replace ZZZZ with the correct update operator
    updates.append(UpdateOne({ "_id": person["_id"] }, { ZZZZ: { "birthday": dateparser.parse(person["birthday"]) } }))
    
    count += 1
                       
    if count == batch_size:
        people_raw.bulk_write(updates)
        updates = []
        count = 0
        
if updates:         
    people_raw.bulk_write(updates)
    count = 0

In [ ]:
# If everything went well this should be zero
people_with_string_birthdays.count()